This script parses the Wilco weekly inventory report and creates a *clean* version of the report. 

The wilco convention for naming files seems to be newstock-{data}T{N}.xlsx. 

The meta file in IN_DIR "lb_inventory_categories.xlsx" has the sku mapping to categories.

The output file name created is *lb_sku_inventory.xlsx*

For running this script initialize the following
```
IN_DIR : Points to the input directory that has the wilco report is stored.
IN_REPORT : Points to the incoming report.
META_DIR: Holds the directory to "lb_inventory_categtories.xlsx"

OUT_DIR : Ponts to the output directory that is used to store the output file
OUT_REPORT : Is the name of the output report.
```


In [17]:
REPORT_DIR = "wilco_reports"
INVENTORY_REPORT = "newstock - 2022-08-05T201013.974.xlsx"

META_DIR = "meta"
CATEGORY_XLSX = "lb_inventory_categories.xlsx"

OUT_DIR = "out"
OUT_REPORT = "lb_sku_inventory.xlsx"

In [11]:
import re
import os
import openpyxl
from pathlib import Path
import lb_wilco_parser as wilco

CATEGORY_XLSX = Path(META_DIR, CATEGORY_XLSX)
IN_XLSX = Path(REPORT_DIR, INVENTORY_REPORT)

if not os.path.exists(OUT_DIR):
    print("Creating output directory")
    os.mkdir(OUT_DIR)
    
OUT_XLSX = Path(OUT_DIR, OUT_REPORT)
print(f"Process {IN_XLSX} to {OUT_XLSX}")


Process wilco_reports/newstock - 2022-08-05T201013.974.xlsx to out/lb_sku_inventory.xlsx


In [12]:
wb = openpyxl.load_workbook(CATEGORY_XLSX)
sheet = wb.active
categories = wilco.create_categories_database(sheet)

Found 88 matching categories


In [13]:
wb = openpyxl.load_workbook(IN_XLSX)
sheet = wb.active
print(sheet)

<Worksheet "newstock">


In [14]:
header_row_idx = wilco.extract_wilco_header_row_index(sheet)
print(header_row_idx)

7


In [15]:
inventory = wilco.create_inventory_database(sheet, header_row_idx)
for item in inventory:
    sku = item[0]
    item_type = categories[sku][0]
    category = categories[sku][1]
    item.append(item_type)
    item.append(category)

In [16]:
wilco.save_inventory_database_to_xlsx(OUT_XLSX, inventory)